In [ ]:
!pip install cassandra-driver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 45.8 MB/s eta 0:00:00


In [ ]:

#Import the necessary libraries
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

In [ ]:
# This secure connect bundle is autogenerated when you download your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-mohandatabase.zip'
}

# This token JSON file is autogenerated when you download your token,
# if yours is different update the file name below
with open("mohandatabase-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

if session:
  print('Connected!')
else:
  print("An error occurred.")


Connected!


In [ ]:
import pandas as pd
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Load the CSV data into a pandas dataframe
url = "https://raw.githubusercontent.com/gchandra10/filestorage/main/sales_100.csv"
data = pd.read_csv(url)

# Display the first few rows of the data to verify
print(data.head())


                         Region           Country  Item Type Sales Channel  \
0            Sub-Saharan Africa      South Africa     Fruits       Offline   
1  Middle East and North Africa           Morocco    Clothes        Online   
2         Australia and Oceania  Papua New Guinea       Meat       Offline   
3            Sub-Saharan Africa          Djibouti    Clothes       Offline   
4                        Europe          Slovakia  Beverages       Offline   

  Order Priority  Order Date   Order ID   Ship Date  UnitsSold  UnitPrice  \
0              M   7/27/2012  443368995   7/28/2012       1593       9.33   
1              M   9/14/2013  667593514  10/19/2013       4611     109.28   
2              M   5/15/2015  940995585    6/4/2015        360     421.89   
3              H   5/17/2017  880811536    7/2/2017        562     109.28   
4              L  10/26/2016  174590194   12/4/2016       3973      47.45   

   UnitCost  TotalRevenue  TotalCost  TotalProfit  
0      6.92     

In [ ]:
session.set_keyspace('mohan')  # Replace 'your_keyspace' with the correct name
print(f"Connected to keyspace: {session.keyspace}")

Connected to keyspace: mohan


In [ ]:
# Query to fetch all tables from the keyspace 'mohan'
query = "SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'mohan'"

# Execute the query and display the table names
print("Tables available in the 'mohan' keyspace:")
for record in session.execute(query):
    print(f"- {record.table_name}")


Tables available in the 'mohan' keyspace:


In [ ]:

# Get a list of all tables in the keyspace
rows = session.execute("SELECT table_name FROM system_schema.tables WHERE keyspace_name = 'mohan'")

In [ ]:
# Define the query to create the 'sales_bronze' table
create_table_query = """
CREATE TABLE IF NOT EXISTS bronze_sales_data (
    region TEXT,
    country TEXT,
    item_type TEXT,
    sales_channel TEXT,
    order_priority TEXT,
    order_date TEXT,
    order_id UUID PRIMARY KEY,
    ship_date TEXT,
    units_sold INT,
    unit_price FLOAT,
    unit_cost FLOAT,
    total_revenue FLOAT,
    total_cost FLOAT,
    total_profit FLOAT
);
"""

# Execute the query to create the table
session.execute(create_table_query)

# Confirmation message
print("The 'bronze_sales_data' table has been successfully created.")


The 'bronze_sales_data' table has been successfully created.


In [ ]:
# Load your CSV file into a DataFrame
csv_file = '/content/sales_100.csv'
df = pd.read_csv(csv_file)
print(df)


                          Region           Country  Item Type Sales Channel  \
0             Sub-Saharan Africa      South Africa     Fruits       Offline   
1   Middle East and North Africa           Morocco    Clothes        Online   
2          Australia and Oceania  Papua New Guinea       Meat       Offline   
3             Sub-Saharan Africa          Djibouti    Clothes       Offline   
4                         Europe          Slovakia  Beverages       Offline   
..                           ...               ...        ...           ...   
94         Australia and Oceania       New Zealand  Beverages        Online   
95            Sub-Saharan Africa           Liberia  Baby Food       Offline   
96                          Asia      Turkmenistan       Meat        Online   
97            Sub-Saharan Africa            Malawi       Meat       Offline   
98         Australia and Oceania           Vanuatu     Cereal       Offline   

   Order Priority  Order Date   Order ID   Ship Dat

In [ ]:
import pandas as pd
import uuid

# Load data from CSV
df = pd.read_csv('/content/sales_100.csv')

# Insert data into the Bronze table
for _, row in df.iterrows():
    # Generate a unique order_id for each record
    order_id = uuid.uuid4()

    # Prepare the data to be inserted
    values = (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'],
        row['Order Priority'], row['Order Date'], order_id,
        row['Ship Date'], row['UnitsSold'], row['UnitPrice'],
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    )

    # Insert data into the Bronze table
    session.execute("""
        INSERT INTO bronze_sales_data (region, country, item_type, sales_channel, order_priority, order_date, order_id, ship_date, units_sold, unit_price, unit_cost, total_revenue, total_cost, total_profit)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, values)

    # Query the data that was just inserted
    inserted_row = session.execute("""
        SELECT * FROM bronze_sales_data WHERE order_id = %s
    """, (order_id,)).one()

    # Print the inserted data
    print(f"Inserted data for Order ID: {inserted_row.order_id}")
    print(f"Region: {inserted_row.region}, Country: {inserted_row.country}, Item Type: {inserted_row.item_type}")
    print(f"Sales Channel: {inserted_row.sales_channel}, Order Priority: {inserted_row.order_priority}")
    print(f"Order Date: {inserted_row.order_date}, Ship Date: {inserted_row.ship_date}")
    print(f"Units Sold: {inserted_row.units_sold}, Unit Price: {inserted_row.unit_price}")
    print(f"Unit Cost: {inserted_row.unit_cost}, Total Revenue: {inserted_row.total_revenue}")
    print(f"Total Cost: {inserted_row.total_cost}, Total Profit: {inserted_row.total_profit}")
    print("-" * 80)

print("Data insertion into 'bronze_sales_data' is complete.")


Inserted data for Order ID: 77e99330-821c-41d6-a13c-fe4b2e83aca0
Region: Sub-Saharan Africa, Country: South Africa, Item Type: Fruits
Sales Channel: Offline, Order Priority: M
Order Date: 7/27/2012, Ship Date: 7/28/2012
Units Sold: 1593, Unit Price: 9.329999923706055
Unit Cost: 6.920000076293945, Total Revenue: 14862.6904296875
Total Cost: 11023.5595703125, Total Profit: 3839.1298828125
--------------------------------------------------------------------------------
Inserted data for Order ID: 6ce517e8-22ff-4397-901c-c31fea1cd419
Region: Middle East and North Africa, Country: Morocco, Item Type: Clothes
Sales Channel: Online, Order Priority: M
Order Date: 9/14/2013, Ship Date: 10/19/2013
Units Sold: 4611, Unit Price: 109.27999877929688
Unit Cost: 35.84000015258789, Total Revenue: 503890.09375
Total Cost: 165258.234375, Total Profit: 338631.84375
--------------------------------------------------------------------------------
Inserted data for Order ID: 1e07b92a-3266-4c44-bf36-80bf47213

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/sales_100.csv')

# Check for any null values in the entire DataFrame
null_values = df.isnull().sum()

# Print the columns with the count of null values
print("Columns with null values:")
print(null_values[null_values > 0])

# If you want to check if there are any null values in the entire dataset
if df.isnull().values.any():
    print("There are missing (null) values in the dataset.")
else:
    print("There are no missing (null) values in the dataset.")


Columns with null values:
Series([], dtype: int64)
There are no missing (null) values in the dataset.


In [ ]:
# Create the 'sales_silver' table with a slightly different approach
create_table_query = """
    CREATE TABLE IF NOT EXISTS sales_silver (
        order_id UUID PRIMARY KEY,
        region TEXT,
        country TEXT,
        item_type TEXT,
        total_profit FLOAT
    );
"""

# Execute the table creation query
try:
    session.execute(create_table_query)
    print("Table 'sales_silver' created successfully.")
except Exception as e:
    print(f"Error creating table 'sales_silver': {e}")


Table 'sales_silver' created successfully.


In [ ]:
rows = session.execute("SELECT order_id, region, country, item_type, total_profit FROM bronze_sales_data;")
for row in rows:
    session.execute("""
        INSERT INTO sales_silver (order_id, region, country, item_type, total_profit)
        VALUES (%s, %s, %s, %s, %s)
    """, (row.order_id, row.region, row.country, row.item_type, row.total_profit))

print("Data successfully inserted into sales_silver.")


Data successfully inserted into sales_silver.


In [ ]:
# Define the CQL query for creating the 'sales_by_region' table
create_sales_by_region_table = """
    CREATE TABLE IF NOT EXISTS sales_by_region (
        region TEXT PRIMARY KEY,
        total_profit FLOAT
    );
"""

# Execute the query to create the table
try:
    session.execute(create_sales_by_region_table)
    print("Table 'sales_by_region' created successfully.")
except Exception as e:
    print(f"Error creating table 'sales_by_region': {e}")


Table 'sales_by_region' created successfully.


In [42]:
import pandas as pd

# Step 1: Load data from sales_bronze
query = "SELECT region, total_profit FROM bronze_sales_data"
rows = session.execute(query)

# Step 2: Convert the fetched data into a Pandas DataFrame for easy manipulation
df = pd.DataFrame(rows)

# Step 3: Perform the aggregation using Pandas
aggregated_data = df.groupby('region', as_index=False)['total_profit'].sum()

# Step 4: Insert aggregated data into the sales_by_region table
for index, row in aggregated_data.iterrows():
    try:
        session.execute("""
            INSERT INTO sales_by_region (region, total_profit)
            VALUES (%s, %s)
        """, (row['region'], row['total_profit']))
        print(f"Successfully inserted data for region: {row['region']}")
    except Exception as e:
        print(f"Error inserting data for region {row['region']}: {e}")

print("Data successfully inserted into sales_by_region.")


Successfully inserted data for region: Asia
Successfully inserted data for region: Australia and Oceania
Successfully inserted data for region: Central America and the Caribbean
Successfully inserted data for region: Europe
Successfully inserted data for region: Middle East and North Africa
Successfully inserted data for region: North America
Successfully inserted data for region: Sub-Saharan Africa
Data successfully inserted into sales_by_region.


In [43]:
# Define the query for creating the 'sales_by_item_type' table
create_sales_by_item_type_table = """
    CREATE TABLE IF NOT EXISTS sales_by_item_type (
        item_type TEXT PRIMARY KEY,
        total_units_sold INT
    );
"""

# Execute the query to create the table and handle any potential errors
try:
    session.execute(create_sales_by_item_type_table)
    print("Table 'sales_by_item_type' created successfully.")
except Exception as e:
    print(f"Error creating table 'sales_by_item_type': {e}")


Table 'sales_by_item_type' created successfully.


In [44]:
import pandas as pd

# Step 1: Fetch data from the sales_bronze table
query = "SELECT item_type, units_sold FROM bronze_sales_data"
rows = session.execute(query)

# Step 2: Load the fetched data into a Pandas DataFrame
df = pd.DataFrame(rows)

# Step 3: Perform aggregation by 'item_type' and sum up 'units_sold'
aggregated_data = df.groupby('item_type')['units_sold'].sum().reset_index()

# Step 4: Insert aggregated data into the sales_by_item_type table
for _, row in aggregated_data.iterrows():
    insert_query = """
        INSERT INTO sales_by_item_type (item_type, total_units_sold)
        VALUES (%s, %s)
    """
    try:
        session.execute(insert_query, (row['item_type'], row['units_sold']))
        print(f"Inserted data for item_type: {row['item_type']}")
    except Exception as e:
        print(f"Error inserting data for item_type {row['item_type']}: {e}")

print("Data successfully inserted into sales_by_item_type.")


Inserted data for item_type: Baby Food
Inserted data for item_type: Beverages
Inserted data for item_type: Cereal
Inserted data for item_type: Clothes
Inserted data for item_type: Cosmetics
Inserted data for item_type: Fruits
Inserted data for item_type: Household
Inserted data for item_type: Meat
Inserted data for item_type: Office Supplies
Inserted data for item_type: Personal Care
Inserted data for item_type: Snacks
Inserted data for item_type: Vegetables
Data successfully inserted into sales_by_item_type.


In [45]:
# Define the CQL query for creating the 'monthly_sales_summary' table
create_monthly_sales_summary_table = """
    CREATE TABLE IF NOT EXISTS monthly_sales_summary (
        order_date TEXT PRIMARY KEY,
        total_revenue FLOAT
    );
"""

# Execute the query to create the table and handle any potential errors
try:
    session.execute(create_monthly_sales_summary_table)
    print("Table 'monthly_sales_summary' created successfully.")
except Exception as e:
    print(f"Error creating table 'monthly_sales_summary': {e}")


Table 'monthly_sales_summary' created successfully.


In [47]:
import pandas as pd

# Step 1: Fetch data from the sales_bronze table
query = "SELECT order_date, total_revenue FROM bronze_sales_data"
rows = session.execute(query)

# Step 2: Load the data into a Pandas DataFrame
df = pd.DataFrame(rows)

# Step 3: Perform the aggregation using groupby and sum the 'total_revenue' by 'order_date'
aggregated_data = df.groupby('order_date')['total_revenue'].sum().reset_index()

# Step 4: Insert aggregated data into the monthly_sales_summary table
insert_query = """
    INSERT INTO monthly_sales_summary (order_date, total_revenue)
    VALUES (%s, %s)
"""
for _, row in aggregated_data.iterrows():
    try:
        session.execute(insert_query, (row['order_date'], row['total_revenue']))
        print(f"Inserted data for order_date: {row['order_date']}")
    except Exception as e:
        print(f"Error inserting data for order_date {row['order_date']}: {e}")

print("Data successfully inserted into monthly_sales_summary.")


Inserted data for order_date: 1/15/2016
Inserted data for order_date: 1/18/2013
Inserted data for order_date: 1/19/2011
Inserted data for order_date: 1/21/2016
Inserted data for order_date: 1/27/2013
Inserted data for order_date: 1/28/2013
Inserted data for order_date: 10/10/2012
Inserted data for order_date: 10/11/2012
Inserted data for order_date: 10/17/2016
Inserted data for order_date: 10/2/2010
Inserted data for order_date: 10/2/2011
Inserted data for order_date: 10/20/2014
Inserted data for order_date: 10/22/2012
Inserted data for order_date: 10/26/2016
Inserted data for order_date: 10/28/2015
Inserted data for order_date: 10/7/2012
Inserted data for order_date: 10/9/2012
Inserted data for order_date: 11/1/2011
Inserted data for order_date: 11/11/2015
Inserted data for order_date: 11/11/2016
Inserted data for order_date: 11/13/2014
Inserted data for order_date: 11/15/2011
Inserted data for order_date: 11/17/2012
Inserted data for order_date: 11/19/2011
Inserted data for order_dat

In [48]:
# List of Gold tables to query
gold_tables = ["sales_by_region", "sales_by_item_type", "monthly_sales_summary"]

# Iterate through each table and fetch its data
for table in gold_tables:
    try:
        # Execute the query to get all rows from the table
        result_set = session.execute(f"SELECT * FROM {table}")

        # Print the results for each table
        print(f"Results from {table}:")
        if result_set:
            for record in result_set:
                print(record)
        else:
            print(f"No data found in {table}")

    except Exception as e:
        print(f"Error retrieving data from {table}: {e}")

print("Data from Gold tables fetched successfully.")


Results from sales_by_region:
Row(region='Australia and Oceania', total_profit=10460820.0)
Row(region='Europe', total_profit=33801844.0)
Row(region='Middle East and North Africa', total_profit=19542786.0)
Row(region='Central America and the Caribbean', total_profit=12756900.0)
Row(region='Asia', total_profit=20249688.0)
Row(region='Sub-Saharan Africa', total_profit=22955676.0)
Row(region='North America', total_profit=4213864.5)
Results from sales_by_item_type:
Row(item_type='Household', total_units_sold=172920)
Row(item_type='Office Supplies', total_units_sold=128442)
Row(item_type='Vegetables', total_units_sold=22104)
Row(item_type='Snacks', total_units_sold=43131)
Row(item_type='Personal Care', total_units_sold=117135)
Row(item_type='Meat', total_units_sold=151311)
Row(item_type='Fruits', total_units_sold=197760)
Row(item_type='Beverages', total_units_sold=135618)
Row(item_type='Cereal', total_units_sold=137328)
Row(item_type='Cosmetics', total_units_sold=197121)
Row(item_type='Baby 

In [51]:
# Query for Bronze table
bronze_query = "SELECT * FROM bronze_sales_data;"
bronze_data = session.execute(bronze_query)

print("------ Bronze Table Data ------")
for row in bronze_data:
    print(row)

# Query for Silver table
silver_query = "SELECT * FROM sales_silver;"
silver_data = session.execute(silver_query)

print("\n------ Silver Table Data ------")
for row in silver_data:
    print(row)



------ Bronze Table Data ------
Row(order_id=UUID('0f529955-6566-4522-98f5-1e9e01788ae3'), country='Malawi', item_type='Meat', order_date='3/1/2016', order_priority='H', region='Sub-Saharan Africa', sales_channel='Offline', ship_date='3/12/2016', total_cost=1857730.875, total_profit=291376.8125, total_revenue=2149107.75, unit_cost=364.69000244140625, unit_price=421.8900146484375, units_sold=5094)
Row(order_id=UUID('50209ab1-8018-4fa4-95ba-989f0a0741e7'), country='France', item_type='Cosmetics', order_date='12/7/2015', order_priority='M', region='Europe', sales_channel='Online', ship_date='1/18/2016', total_cost=1516254.125, total_profit=1001143.4375, total_revenue=2517397.5, unit_cost=263.3299865722656, unit_price=437.20001220703125, units_sold=5758)
Row(order_id=UUID('0d263c61-a5d0-48b7-ab5c-cb47a918c5ba'), country='India', item_type='Snacks', order_date='10/10/2012', order_priority='L', region='Asia', sales_channel='Online', ship_date='11/20/2012', total_cost=521206.5625, total_profi